In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [3]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  10 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코코아&start=91']


100%|██████████| 151/151 [00:00<?, ?it/s]


In [4]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 51/51 [00:12<00:00,  3.96it/s]

검색된 기사 갯수: 총  100 개

[뉴스 제목]
['코코아값 올들어 21% 상승…초콜릿 가격도 오를 수도', '\n\t\t\t‘이잘생’ 신혜선X안보현X하윤경X안동구, 따뜻한 코코아 같은 힐링 온다[종합]\n\t\t', "'다이어트의 적'이라는 코코아...'이렇게' 먹으면 '혈관 건강'에 좋다?", '[3가지 경제] 비빔밥 1만 원 시대', '코코아, 왜 혈관 건강을 도울까?', '치솟는 초콜릿값…카카오 가격 7년 만에 최고치', '커피값 또 들썩일라…‘엘니뇨’ 이상기후에 심상찮은 작황', '[모던 경성]1930년대 ‘차당(茶黨)의 여왕’이 꼽은 커피 맛집은 어디?', "'인스턴트 커피' 가격 실화냐…'도난방지 장치 판매' 진풍경", "檢, 김남국 자본시장법 적용 겨눈 듯…'위메이드 허위 공시' 살핀다", '체지방 증가, 활력 저하까지...부족한 테스토스테론 보충하는 식품은?', "초콜릿 주원료 '코코아 빈' 일주일 먹었더니… 나쁜 콜레스테롤 '뚝'", '일주일 코코아 먹었더니 생긴 신기한 신체 변화', "'코코아' 딱 일주일 마셨는데…놀라운 일 벌어졌다", "추울 때 먹는 코코아… '이곳' 건강에 좋다?", '[리빙포인트] 심혈관 건강에 좋은 코코아', '코코아, 아동노동 그리고 ‘공급망 실사법’[안치용의 까칠한 ESG 이야기](8)', "코코아·소다·에이드 등장, '니케' 메이드 카페 열린다", "코코아의 '이 성분' 심혈관질환 사망 위험 낮춰", '클레이튼 생태계 넓히는 카카오 크러스트…‘코코아 파이낸스’에 투자', 'None', '피자의 고향 이탈리아서 “마르게리따 못 만들겠다” 무슨일이', '코코아·다크 초콜릿 플라바놀의 혈류 개선 효과 증명', '동서식품, 달콤한 코코아와 산뜻한 페퍼민트 ‘미떼 핫초코 민트초코’ 선보여', '오리온, 30년 전통 초코칩쿠키 \'프랑스산 코코아\' 포기..."원가 절감"', '이 겨울, 코코아 대신 녹차와 돼지감자차를 택한 이유', '운동 전 코코아 한 잔, 최고의 간식 (연구)', '“코코아, 자주 많이 먹으니 더

In [5]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('.\\DATA\\{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  51
